In [1]:
import os
import pandas as pd
import json
from utils import normalize_trait, judge_and_remove_biased_features

utils.py has been loaded


In [2]:
source_root = "/home/techt/AI4Science_gold_standard/data/preprocessed/gold"
dest_root = "./gold_subset"

## check Xena data omission

In [ ]:
s_ls = []
for t in os.listdir(dest_root):
    s_path = os.path.join(source_root, t)
    if not os.path.isdir(s_path):
        f_ls = os.listdir(os.path.join(dest_root, t))
        if len(f_ls) == 0 or (len(f_ls) == 1 and f_ls[0].endswith('.json')):
            pass
        else:
            # print(f"Strange. Dir {s_path} does not exist!")
            s_ls.append(t)
    else:
        for f in os.listdir(s_path):
            if f.endswith('Xena.csv'):
                source_fpath = os.path.join(s_path, f)
                if os.path.isfile(os.path.join(dest_root, t, 'Xena.csv')):
                    print(f"{os.path.join(dest_root, t, 'Xena.csv')} exist corresponding to {source_fpath}")
                else:
                    print(f"No! {os.path.join(dest_root, t, 'Xena.csv')} not exist corresponding to {source_fpath}")

In [ ]:
sorted(s_ls)

## Fix table abnormality
delete table with missing genetic data, and unify trait name as header. Creat json files 'cohort_info.json'.

In [3]:
all_traits = pd.read_csv("all_traits.csv")["Trait"].tolist()
all_nm_traits = [normalize_trait(at) for at in all_traits]

In [ ]:
sorted(all_nm_traits)

In [ ]:
len(set(all_nm_traits))

In [ ]:
sorted(all_traits)

In [ ]:
# 检查路径和列名里的trait_name，进行normalize
# 没有gene的table去掉
# 创建或修改json文件

In [ ]:
for trait in os.listdir(dest_root):
    if trait not in all_nm_traits:
        if trait in all_traits:
            print(f"'{trait}' trait needs to be normalized ")
        else:
            print(f"'{trait}' strange trait")
        continue
    trait_dir = os.path.join(dest_root, trait)
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            df = pd.read_csv(fp)
            if 'Unnamed: 0' in df.columns:
                print(f"REDUNDANT COLUMN in '{fp}'")
                df = df.drop(columns = ['Unnamed: 0'], errors="ignore")
            header = df.columns[0]
            if header not in all_traits:
                if header in all_nm_traits:
                    org_trait = all_traits[all_nm_traits.index(header)]
                    print(f"The header {header} should be replaced by {org_trait} in '{fn}'")
                    df.columns = [org_trait] + df.columns[1:].tolist()
                    df.to_csv(fp, index=False)
                    print('This cohort updated')
                else:
                    print(f"Strange header {header}")
            elif header == "Lupus-(Systemic-Lupus-Erythematosus":
                df.columns = ["Lupus (Systemic Lupus Erythematosus)"] + df.columns[1:].tolist()
                df.to_csv(fp, index=False)
                print("The cohort for 'Lupus' updated")

            if len(df.columns) <= 4:
                print(f"NO GENE DATA in '{fp}'")
                os.remove(fp)
                print("File removed successfully")


In [4]:
for trait in os.listdir(dest_root):
    if trait != 'Lung-Cancer':
        continue
    else:
        print("Haha lung cancer")
    if trait not in all_nm_traits:
        if trait in all_traits:
            print(f"'{trait}' trait needs to be normalized ")
        else:
            print(f"'{trait}' strange trait")
        continue
    trait_dir = os.path.join(dest_root, trait)
    meta = dict()
    for fn in os.listdir(trait_dir):
        fp = os.path.join(trait_dir, fn)
        if fn.endswith(".csv") and (fn.startswith("GSE") or fn.startswith("Xena")):
            cohort = fn[:-4]
            df = pd.read_csv(fp)
            header = df.columns[0]
            assert header in all_traits
            trait_type = 'binary' if len(df[header].unique()) <= 2 else 'continuous'
            # May mistakenly judge (28, 28, 28, 28, ..., 100) as unbiased
            # May later try len(unique_values) == 2 and set(unique_values) <= {0, 1}
            trait_biased, _ = judge_and_remove_biased_features(df, header, trait_type)
            has_age = 'Age' in df.columns or 'age' in df.columns
            has_gender = 'Gender' in df.columns or 'gender' in df.columns
            sample_size = len(df)
            if not trait_biased:
                meta[cohort] = {"is_usable": True, "is_available": True, "is_biased": False, "has_age": has_age, "has_gender": has_gender, "sample_size": sample_size, "note": ""}
    js_path = os.path.join(trait_dir, "cohort_info.json")
    if os.path.exists(js_path):
        os.remove(js_path)
    with open(js_path, 'w') as file:
        json.dump(meta, file)


Haha lung cancer
For the feature 'Lung Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



In [ ]:
from utils import read_json_to_dataframe, filter_and_rank_cohorts
b = read_json_to_dataframe('gold_subset/Creutzfeldt-Jakob-Disease/cohort_info.json')
b

In [ ]:
b.empty

In [ ]:
filter_and_rank_cohorts('gold_subset/Creutzfeldt-Jakob-Disease/cohort_info.json')

In [ ]:
#'./gold_subset/Multiple-sclerosis/GSE215450.csv'

In [ ]:
pd.read_csv('./gold_subset/Multiple-sclerosis/GSE215450.csv')

In [ ]:
pd.read_csv('./gold_subset/Multiple-sclerosis/GSE171950.csv')

In [ ]:
len(df['Glucocorticoid Sensitivity'].unique())

In [ ]:
df['Glucocorticoid Sensitivity'].unique() in [1,0]